In [2]:
%load_ext autoreload
%autoreload 2

import taxi_demand_predictor.config as cfg
import taxi_demand_predictor.data as data
import taxi_demand_predictor.preprocessing as preprocessing 
import taxi_demand_predictor.model as model
from taxi_demand_predictor.paths import MODELS_DIR
from sklearn.metrics import mean_absolute_error
import pandas as pd
import optuna
import hopsworks
import joblib
from hsml.schema import Schema 
from hsml.model_schema import ModelSchema
from hsfs.client.exceptions import RestAPIError

2024-08-16 04:32:36,814 - INFO - Folder "data" ensured at "/Users/borja/Documents/Somniumrema/projects/ml/taxi_demand_predictor/data"
2024-08-16 04:32:36,814 - INFO - Folder "raw" ensured at "/Users/borja/Documents/Somniumrema/projects/ml/taxi_demand_predictor/data/raw"
2024-08-16 04:32:36,814 - INFO - Folder "bronze" ensured at "/Users/borja/Documents/Somniumrema/projects/ml/taxi_demand_predictor/data/bronze"
2024-08-16 04:32:36,815 - INFO - Folder "silver" ensured at "/Users/borja/Documents/Somniumrema/projects/ml/taxi_demand_predictor/data/silver"
2024-08-16 04:32:36,816 - INFO - Folder "gold" ensured at "/Users/borja/Documents/Somniumrema/projects/ml/taxi_demand_predictor/data/gold"
2024-08-16 04:32:36,817 - INFO - Folder "models" ensured at "/Users/borja/Documents/Somniumrema/projects/ml/taxi_demand_predictor/models"


In [3]:
# Connect to the project
project = hopsworks.login(
    project=cfg.HOPSWORKS_PROJECT_NAME,
    api_key_value=cfg.HOPSWORKS_API_KEY
)

# Connect to the feature store
feature_store = project.get_feature_store()

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/933012
Connected. Call `.close()` to terminate connection gracefully.


In [18]:
# Connect to the feature group
feature_group = feature_store.get_feature_group(
    cfg.FEATURE_GROUP_METADATA['name'],
    cfg.FEATURE_GROUP_METADATA['version']
)

In [19]:
# Create a feature view in hopsworks
try:
    # Try getting the feature view first
    feature_view = feature_store.get_feature_view(
        cfg.FEATURE_VIEW_METADATA['name'],
        cfg.FEATURE_VIEW_METADATA['version']
    )
    print("The feature view already exists")
except RestAPIError as e:
    if e.response.status_code == 404:  # Specific HTTP status code for not found
        # Ensure the query selects all features from the feature group
        query = feature_group.select_all()
        
        # Create the feature view if it does not exist
        feature_view = feature_store.create_feature_view(
            **cfg.FEATURE_VIEW_METADATA,
            query=query
        )
        print("Feature view created successfully")
    else:
        # Re-raise the exception if it's a different error
        raise

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/933012/fs/926787/fv/view_group_features_hourly_2023_03_29_to_2024_05_24/version/1
Feature view created successfully


In [20]:
# Query the feature view
ts_data, _ = feature_view.training_data(
    description="ts data hourly taxi rides",
)

# sort values to see the features in the feature store 
ts_data.sort_values(by=['pickup_location_id', 'pickup_time'], inplace=True)

# Print values
ts_data

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (10.09s) 


2024-08-16 04:46:11,540 - WARNING - VersionWarning: Incremented version to `1`.



,pickup_time,pickup_location_id,ride_count,pickup_ts
19058,2024-03-29 02:00:00+00:00,1,0.0,1711677600000
145858,2024-03-29 03:00:00+00:00,1,0.0,1711681200000
189196,2024-03-29 04:00:00+00:00,1,0.0,1711684800000
185827,2024-03-29 05:00:00+00:00,1,0.0,1711688400000
213666,2024-03-29 06:00:00+00:00,1,0.0,1711692000000
...,...,...,...,...
280962,2024-05-23 20:00:00+00:00,265,5.0,1716494400000
240181,2024-05-23 21:00:00+00:00,265,2.0,1716498000000
328514,2024-05-23 22:00:00+00:00,265,3.0,1716501600000
141956,2024-05-23 23:00:00+00:00,265,8.0,1716505200000


In [21]:
# Generate features and target
features_and_target = data.generate_training_set(ts_data, start_position = 0, n_features = 24*28, step_size = 23, pickup_location_id = None, target_col = 'ride_count')

# Make a copy of the features and target
features_and_target_copy = features_and_target.copy()

# Print the features and target
print(f"{features_and_target.shape= }")

features_and_target.shape= (7830, 675)


In [22]:
# Show features and target dataframe
features_and_target.head()

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_time,pickup_location_id,rides_next_hour
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2024-04-26 02:00:00,1,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,2.0,0.0,0.0,0.0,0.0,2024-04-27 01:00:00,1,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,2.0,2.0,1.0,0.0,0.0,0.0,0.0,2024-04-28 00:00:00,1,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.0,5.0,2.0,10.0,1.0,1.0,0.0,2024-04-28 23:00:00,1,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,1.0,...,2.0,0.0,0.0,2.0,1.0,1.0,0.0,2024-04-29 22:00:00,1,0.0


In [27]:
# Get the maximum and minimum dates from the 'pickup_time' column
max_date = ts_data['pickup_time'].max()
min_date = ts_data['pickup_time'].min()

# Calculate the difference between the two dates
date_difference = max_date - min_date

# Extract the number of days from the difference
number_of_days = date_difference.days

print(f"Number of days between {min_date} and {max_date}: {number_of_days} days")

Number of days between 2024-03-29 02:00:00+00:00 and 2024-05-24 00:00:00+00:00: 55 days


In [28]:
# Cutoff date leaving out two months of data
days_from_end_date = 14

# Define the cutoff date
cutoff_date = ts_data['pickup_time'].max() - pd.Timedelta(days=days_from_end_date)

# Convert cutoff date to string
cutoff_date_str = cutoff_date.strftime('%Y-%m-%d %H:%M:%S')

# Split the data into training and validation
X_train, y_train, X_test, y_test = preprocessing.split_data(features_and_target_copy, datetime_column = 'pickup_time', cutoff_datetime = cutoff_date_str, target_column = 'rides_next_hour')

# Print shapes of train and test data
print(f"{X_train.shape= }, {y_train.shape= }, {X_test.shape= }, {y_test.shape= }")

X_train.shape= (3915, 674), y_train.shape= (3915,), X_test.shape= (3915, 674), y_test.shape= (3915,)


In [30]:
# Create an optuna study
study = optuna.create_study(direction="minimize")

# Optimize the model hyperparameters
study.optimize(lambda trial: model.objective(trial, X_train, y_train, model.get_pipeline, mean_absolute_error, 3), n_trials=20)

[I 2024-08-16 04:50:02,436] A new study created in memory with name: no-name-c46f1931-6417-425a-b88b-c0aa62990210
2024-08-16 04:50:02,494 - WARNING - SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

2024-08-16 04:50:03,297 - WARNING - SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

2024-08-16 04:50:03,324 - WARNING - SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://panda

In [31]:
# Get the best hyperparameters from Optuna optimization
best_params = study.best_trial.params

# Print the best hyperparameters
print(f'{best_params=}')

# train the model with the best hyperparameters on the entire training data
pipeline = model.get_pipeline(**best_params)
pipeline.fit(X_train, y_train)

best_params={'num_leaves': 167, 'feature_fraction': 0.37055430922189314, 'bagging_fraction': 0.6934049444332455, 'min_child_samples': 19}


Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function period_avg_4_weeks at 0x13f77bec0>)),
                ('temporalfeaturesengineer', TemporalFeaturesEngineer()),
                ('columndropper',
                 ColumnDropper(columns_to_drop=['pickup_location_id'])),
                ('lgbmregressor',
                 LGBMRegressor(bagging_fraction=0.6934049444332455,
                               feature_fraction=0.37055430922189314,
                               min_child_samples=19, num_leaves=167))])

In [32]:
# Compute the predictions
predictions = pipeline.predict(X_test)

# Calculate the test error
test_mae = mean_absolute_error(y_test, predictions)

# Print the test error
print(f"{test_mae = }")

test_mae = 3.066910298242691


In [34]:
# Serialize the model to disk
joblib.dump(pipeline, MODELS_DIR / 'model.pkl')

['/Users/borja/Documents/Somniumrema/projects/ml/taxi_demand_predictor/models/model.pkl']

In [35]:
# Define input and output schema for the model
input_schema = Schema(X_train)
output_schema = Schema(y_train)

# Define model schema
model_schema = ModelSchema(input_schema, output_schema)

In [37]:
# Instantiate the pointer to the model registry
model_registry = project.get_model_registry()

# Register the model
model = model_registry.sklearn.create_model( # sklearn create model
    name='taxi_demand_predictor',            # assign a name to the model
    model_schema=model_schema,               # Define model schema
    metrics={'mae': test_mae},               # Define metrics
    description='LightGBM demand predictor', # Describe the model
    input_example=X_train.sample()           # Provide a sample of the input
)

# Register the model
model.save(str(MODELS_DIR / 'model.pkl'))

Connected. Call `.close()` to terminate connection gracefully.


  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/1423483 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/3408 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/58109 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/933012/models/taxi_demand_predictor/4


Model(name: 'taxi_demand_predictor', version: 4)